# AnthropicToolCallState

## Example: Create TOOL_USE state using AnthropicToolUseState

This is actually 2 actions within the same state. The first action is to call the tool and the second action is to pass the result to the LLM and return a response.

```lua
stateDiagram-v2
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```


In [1]:
import os
os.environ["LOG_LEVEL"]="WARNING"

from gai.lib.config import config_helper

llm_config = config_helper.get_client_config("sonnet-4")

In [2]:
from gai.asm import AgenticStateMachine
from gai.messages import Monologue
from gai.mcp.client import McpAggregatedClient

monologue = Monologue()
monologue.add_user_message("What time is it now?")
monologue.add_assistant_message(
    [
        {
            "citations": None,
            "text": "I'll get the current time for you.",
            "type": "text",
        },
        {
            "id": "toolu_012CyVmXRfMRzxoYBtmoeF1s",
            "input": {"format": "YYYY-MM-DD HH:mm:ss"},
            "name": "current_time",
            "type": "tool_use",
        },
    ]
)

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_USE
    TOOL_USE--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_USE": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicToolUseState",
                "title": "TOOL_USE",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
        monologue=monologue,
    )
    fsm.restart()

## Step 2: INIT --> TOOL_CALL

await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if (isinstance(chunk,str)):
        print(chunk, end='', flush=True)
print("\n\n")

## Step 4: Print the state history
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

The current time is **2025-08-04 01:25:32** (UTC).

If you'd like to know the time in a specific timezone, please let me know which timezone you're interested in!



1754270730.0613885 User > What time is it now?
1754270730.0615668 Assistant > [{'citations': None, 'text': "I'll get the current time for you.", 'type': 'text'}, {'id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'input': {'format': 'YYYY-MM-DD HH:mm:ss'}, 'name': 'current_time', 'type': 'tool_use'}]
1754270732.3484035 User > [{'type': 'tool_result', 'tool_use_id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'content': 'Current UTC time is 2025-08-04 01:25:32, and the time in UTC is 2025-08-04 01:25:32.'}]
1754270734.6111593 Assistant > [{'citations': None, 'text': "The current time is **2025-08-04 01:25:32** (UTC).\n\nIf you'd like to know the time in a specific timezone, please let me know which timezone you're interested in!", 'type': 'text'}]


---

## When LLM has completed its task ("task_completed")

We can continue to resume as long as the agent can continue its task.

In the following case, the previous message does not contain a "tool_use" content, that means the agent has either completed its task or is pending user input.

In [10]:
from gai.asm import AgenticStateMachine
from gai.messages import Monologue
from gai.mcp.client import McpAggregatedClient

monologue = Monologue()
monologue.add_user_message("What time is it now?")
monologue.add_assistant_message(
    [
        {
            "citations": None,
            "text": "## Summary\n\nI have successfully completed ...",
            "type": "text",
        }
    ]
)

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_USE
    TOOL_USE--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_USE": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicToolUseState",
                "title": "TOOL_USE",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
        monologue=monologue,
    )
    fsm.restart()

## Step 2: INIT --> TOOL_CALL

await fsm.run_async()
if not fsm.state_bag.get("streamer"):
    print("Nothing to resume.")
else:
    for chunk in fsm.state_bag["streamer"]:
        if (isinstance(chunk,str)):
            print(chunk, end='', flush=True)
    print("\n\n")

## Step 4: Print the state history

for message in fsm.monologue.list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )
    
# The last message should only contain the task_completed tool call, not the current time tool call.


Nothing to resume.
1754271542.3418617 User > What time is it now?
1754271542.3420205 Assistant > [{'citations': None, 'text': '## Summary\n\nI have successfully completed ...', 'type': 'text'}]


---

## When LLM has interrupted itself ("user_input")

When the LLM interrupts itself, it will call a tool and expect a response from the user.
If the user "continue_async", then the agent will not stream any responses.
Instead, the user will provide a "user_message" with continue_async.



In this scenario, the LLM will interrupt the user with a question and expects a response from the user.

The previous message is an agent message that contains the tool "user_message' and the text "Are you asking for the local time or the UTC time?".

#### i) Answer correctly

- Add "Use SGT" to the user message
- Run it.




In [2]:
from gai.asm import AgenticStateMachine
from gai.messages import Monologue
from gai.mcp.client import McpAggregatedClient

monologue = Monologue()
monologue.add_user_message("What time is it now?")
monologue.add_assistant_message(
    [
        {
            "citations": None,
            "text": "Are you asking for the local time or the UTC time?",
            "type": "text",
        },
        {
            "id": "toolu_012CyVmXRfMRzxoYBtmoeF1s",
            "name": "user_input",
            "input": {},
            "type": "tool_use",
        },
    ]
)

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_USE
    TOOL_USE--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_USE": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicToolUseState",
                "title": "TOOL_USE",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
        monologue=monologue,
    )
    fsm.restart()

## Step 2: INIT --> TOOL_CALL

fsm.user_message = "Use SGT"
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
print("\n\n")

## Step 4: Print the state history

for message in fsm.monologue.list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

# The last message should only contain the task_completed tool call, not the current time tool call.






1754271632.4178703 User > What time is it now?
1754271632.4180486 Assistant > [{'citations': None, 'text': 'Are you asking for the local time or the UTC time?', 'type': 'text'}, {'id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'name': 'user_input', 'input': {}, 'type': 'tool_use'}]
1754271633.7673826 User > [{'type': 'tool_result', 'tool_use_id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'content': 'Use SGT'}]
1754271635.9466374 Assistant > [{'id': 'toolu_01Ube3n76eKSLujmPTtjGay2', 'input': {'format': 'YYYY-MM-DD HH:mm:ss', 'timezone': 'Asia/Singapore'}, 'name': 'current_time', 'type': 'tool_use'}]


#### ii) No answer given

- Do not provide user_message.
- Run it.



In [4]:
from gai.asm import AgenticStateMachine
from gai.asm.agents.tool_use_agent import PendingUserInputError
from gai.messages import Monologue
from gai.mcp.client import McpAggregatedClient

monologue = Monologue()
monologue.add_user_message("What time is it now?")
monologue.add_assistant_message(
    [
        {
            "citations": None,
            "text": "Are you asking for the local time or the UTC time?",
            "type": "text",
        },
        {
            "id": "toolu_012CyVmXRfMRzxoYBtmoeF1s",
            "name": "user_input",
            "input": {},
            "type": "tool_use",
        },
    ]
)

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_USE
    TOOL_USE--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_USE": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicToolUseState",
                "title": "TOOL_USE",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
        monologue=monologue,
    )
    fsm.restart()

## Step 2: INIT --> TOOL_CALL
try:
    await fsm.run_async()
    async for chunk in fsm.state_bag["streamer"]:
        if isinstance(chunk, str):
            print(chunk, end="", flush=True)
    print("\n\n")
    assert False, "Expected PendingUserInputError"
except PendingUserInputError as e:
    print("Expected Error:", e)

## Step 4: Print the state history
for message in fsm.monologue.list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

# Notice that the monologue remains unchanged.


Expected Error: AnthropicToolUseState.run_async: pending user input
1754271741.6188421 User > What time is it now?
1754271741.6190364 Assistant > [{'citations': None, 'text': 'Are you asking for the local time or the UTC time?', 'type': 'text'}, {'id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'name': 'user_input', 'input': {}, 'type': 'tool_use'}]
